In [7]:
import numpy as np
from numpy import linalg as la
import sys
sys.path.append('..')
import data

SEED = 10

np.random.seed(SEED)
RESULTS_PATH = 'results/efficient_alg/' 

In [8]:
tries = 20
N = 20
M = 1
k = N

full_rank_S = 0
rank_N = 0
rank_B = 0
full_rank_A_B = 0
full_rank_A_B2 = 0
for i in range(tries):
    g_params = {'type': data.ER, 'p': 0.2}
    S = data.generate_graph(N, g_params)
    eigvals, Eigvecs = np.linalg.eigh(S)
    if la.matrix_rank(S) == N:
        full_rank_S += 1

    
    # x = Eigvecs[:,:k]@np.random.randn(k, M)
    x = S[:,:k]@np.random.randn(k, M)
    # x = np.eye(N)[:,:k]@np.random.randn(k, M)

    # rho = np.random.multivariate_normal(np.zeros(N), S@S+np.eye(N), size=(M)).T
    # x = np.eye(N)[:,:k]@rho[:k,:]

    XX_kron_I = np.kron(x@x.T, np.eye(N))
    # print('Rank XXI:', la.matrix_rank(XX_kron_I))
    if la.matrix_rank(XX_kron_I) == N*min(N,M):
        rank_N += 1
    
    
    B = np.kron(S@S, np.eye(N)) + np.kron(np.eye(N), S@S) - 2*np.kron(S, S)
    # print('Rank(B):', la.matrix_rank(B))
    if la.matrix_rank(B) == N**2 - N:
        rank_B += 1

    rank_A_B = la.matrix_rank(XX_kron_I + B)
    # print('Rank(A+B)',  rank_A_B)
    if rank_A_B == N**2 and la.matrix_rank(B) == N**2-N:
        full_rank_A_B2 += 1
    if rank_A_B == N**2:
        full_rank_A_B += 1
    

print('Full rank S:', full_rank_S/tries)
print('Rank(XXI)=N:', rank_N/tries)
print('Rank(B) = N**2 - N:', rank_B/tries)
print('Full rank A+B:', full_rank_A_B/tries)
print('Full rank A+B (with full rank B):', full_rank_A_B2/rank_B)


Full rank S: 0.7
Rank(XXI)=N: 1.0
Rank(B) = N**2 - N: 0.9
Full rank A+B: 0.65
Full rank A+B (with full rank B): 0.7222222222222222
